흑백 말 실루엣 이미지 가져오기 \
말 모양의 외곽선(윤곽) 찾아내기 \
면적, 중심점, 둘레, 둥근 정도(원에 가까운지)를 계산하기\
외곽선을 다각형으로 단순화, 볼록껍질(Convex Hull) 그리기

In [6]:
import skimage
import numpy as np
import cv2 as cv

# [1] Horse 이미지 불러오기 (bool 배열)
orig = skimage.data.horse()

# [2] 색 반전: 말 부분은 흰색(255), 배경은 검정(0)으로
# (OpenCV Contour 검출은 전경=흰색, 배경=검정 기준!)
img = 255 - np.uint8(orig) * 255

# [3] 원본 확인
cv.imshow('Horse', img)

# [4] 외곽선 찾기 (Contour)
# cv.RETR_EXTERNAL : 가장 바깥쪽 외곽선만 찾음
# cv.CHAIN_APPROX_NONE : 모든 좌표 저장
contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

# [5] 원본에 외곽선 그리기
img2 = cv.cvtColor(img, cv.COLOR_GRAY2RGB)  # 색상 입혀서 컬러로 변환
cv.drawContours(img2, contours, -1, (255, 0, 255), 2)  # 보라색
cv.imshow('Horse with contour', img2)

# [6] 첫 번째 외곽선 가져오기
contour = contours[0]

# [7] 모멘트 계산 (면적, 중심 등)
m = cv.moments(contour)

# [8] 면적 (Contour로 계산)
area = cv.contourArea(contour)

# [9] 중심 좌표 (Centroid)
cx, cy = m['m10'] / m['m00'], m['m01'] / m['m00']

# [10] 둘레 (경계 길이)
perimeter = cv.arcLength(contour, True)  # True: 닫힌 곡선

# [11] 둥근 정도 (Roundness) = 4πA / P²
roundness = (4.0 * np.pi * area) / (perimeter * perimeter)

# [12] 출력
print('면적 =', area,
      '\n중심점 = (', cx, ',', cy, ')',
      '\n둘레 =', perimeter,
      '\n둥근 정도 =', roundness)

# [13] 근사 다각형과 볼록 껍질 시각화 준비
img3 = cv.cvtColor(img, cv.COLOR_GRAY2BGR)

# [14] 다각형 근사 (Polygon Approximation)
# - 단순화된 선분으로 근사 (epsilon=8)
contour_approx = cv.approxPolyDP(contour, 8, True)
cv.drawContours(img3, [contour_approx], -1, (0, 255, 0), 2)  # 초록색

# [15] 볼록 껍질 (Convex Hull)
hull = cv.convexHull(contour)  # 좌표를 볼록 다각형으로 감쌈

# hull 형태를 OpenCV drawContours에 맞게 reshape
hull = hull.reshape(1, hull.shape[0], hull.shape[2])
cv.drawContours(img3, hull, -1, (0, 0, 255), 2)  # 빨간색

# [16] 결과 시각화 (근사 다각형 + 볼록 껍질)
cv.imshow('Horse with line segments and convex hull', img3)

# [17] 키 입력 대기 & 창 닫기
cv.waitKey()
cv.destroyAllWindows()


면적 = 42390.0 
중심점 = ( 187.72464024534088 , 144.43640402610677 ) 
둘레 = 2296.7291333675385 
둥근 정도 = 0.1009842680321435


| 부분             | 이유                         |
| -------------- | -------------------------- |
| `255-np.uint8(orig)*255`    | 배경/전경 반전 (말 모양 흰색)        |
| `findContours` | 모양 경계점 좌표 찾기               |
| `moments`      | 면적, 중심 계산                  |
| `arcLength`    | 둘레 길이 계산                   |
| `roundness`    | 모양이 원에 가까운지 평가             |
| `approxPolyDP` | 복잡한 윤곽선을 단순화 (선분으로)        |
| `convexHull`   | 울퉁불퉁한 모양을 둘러싸는 가장 작은 볼록 형태 |


In [2]:
print(orig.dtype)   # bool
print(np.unique(orig))  # [False  True]
print(type(contours))
print(len(contours))

bool
[False  True]
<class 'tuple'>
1
